In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import pandas as pd
import rasterio.plot

import overpy
import geopandas as gpd
# import ee
from shapely.geometry import Polygon, box
import os
# import geemap
import json
import multiprocessing



In [2]:
# os.chdir('/Users/yer1k/Documents/Master/Duke/2023 Fall/Capstone/solar_wastewater/30_result/')
os.chdir('/Users/yer1k/Library/CloudStorage/GoogleDrive-iamtsubasaozora@gmail.com/My Drive/Capstone Project/Data/')

In [3]:
api = overpy.Overpass()

In [4]:

# Fetch all wastewater treatment plants within California's boundary (# changed to Texas)
query = f"""
    area[admin_level=4]["name"="Texas"]->.searchArea; 
    (
    way["man_made"="wastewater_plant"](area.searchArea);
    );
    (._;>;);
    out body;
    """

result = api.query(query)

plants = {}

for way in result.ways:
    # Use the name of the plant or its ID if the name is not available
    plant_name = way.tags.get("name", f"Plant_{way.id}")

    # Extract nodes lat and lon without id
    nodes_coords = [(node.lon, node.lat) for node in way.nodes]

    # get rid of "Decimal"  in the coordinates
    nodes_coords = [tuple(map(float, i)) for i in nodes_coords]
    
    plants[plant_name] = nodes_coords

# plants_with_official_name = {key: value for key, value in plants.items() if not key.startswith("Plant_")}


geoms = [Polygon(plants[key]) for key in plants]
df = gpd.GeoDataFrame({'WWTP_name': list(plants.keys()), 'geometry': geoms}, crs="EPSG:4326")
df["centroid"] = df.to_crs('+proj=cea').centroid.to_crs(epsg=4326)



In [5]:
df

,WWTP_name,geometry,centroid
0,Plant_27449381,"POLYGON ((-96.77142 32.72644, -96.76411 32.728...",POINT (-96.77224 32.73390)
1,City of Leander Wastewater Treatment Plant,"POLYGON ((-97.84240 30.58064, -97.84133 30.581...",POINT (-97.84113 30.58042)
2,Plant_29415748,"POLYGON ((-97.82238 30.55487, -97.82204 30.555...",POINT (-97.82171 30.55485)
3,Plant_29660147,"POLYGON ((-97.79693 30.46380, -97.79665 30.463...",POINT (-97.79780 30.46284)
4,Albert H. Ullrich Water Treatment Plant,"POLYGON ((-97.79899 30.29007, -97.79933 30.289...",POINT (-97.79621 30.28999)
...,...,...,...
538,Plant_1202606682,"POLYGON ((-95.48860 30.05945, -95.48851 30.059...",POINT (-95.48691 30.05825)
539,Plant_1211640644,"POLYGON ((-95.56537 30.03827, -95.56554 30.038...",POINT (-95.56586 30.03892)
540,Plant_1215636411,"POLYGON ((-97.66318 30.70890, -97.66259 30.709...",POINT (-97.66279 30.70874)
541,Windemere Wastewater Treatment Plant,"POLYGON ((-97.64107 30.45672, -97.64046 30.456...",POINT (-97.64115 30.45588)


In [6]:
!ls

Archive                         Status Track.gsheet
PPT From Christine              WWTP Lists
Satellite Imagery_Automated RGB tif_bounding_box


In [8]:
# use loop to open the image in tif folder, and plot the bounding box on it
# read all the file names in the folder
for file in os.listdir('tif_bounding_box/texas/'):
    try:
        with rasterio.open(os.path.join('tif_bounding_box/texas/', file)) as src:
            # if file.startswith('download'):
                # continue
            # Read the image data
            img = src.read([1, 2, 3])  # Reads the first three bands for RGB
            # file name without extension
            name = os.path.splitext(file)[0]
            # find the corresponding bounding box
            df_name = df.loc[df['WWTP_name'] == name, :]
            # plot the bounding box
            fig, ax = plt.subplots(figsize=(8,8))
            # Plot the raster
            ax = rasterio.plot.show(img, ax=ax, transform=src.transform)
            # Plot the geometry on the ax axes instance
            df_name.boundary.plot(ax=ax, color='red', linewidth=1.5)
            plt.axis('off')
            # save the figure
            plt.savefig(os.path.join('tif_bounding_box/texas/bounding box', name + '.png'), bbox_inches='tight', pad_inches=0)
            plt.close()
    except:
        print(f"fail to plot {file}")

fail to plot .DS_Store
fail to plot bounding box
